In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from os.path import join, exists, isfile, isdir, abspath, dirname, basename, realpath
from os import makedirs, listdir, pardir, getcwd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
parent_dir = abspath(join(join(getcwd(), pardir), pardir))
data_dir = join(parent_dir, 'data')

In [3]:
graph_fly = nx.read_edgelist(join(data_dir, "DM-HT.txt"), nodetype=str, data=(("weight", float),))
# graph_sworm = nx.read_edgelist(join(data_dir, "CE-HT.txt"), nodetype=str, data=(("weight", float),))

In [4]:
graph_fly.number_of_nodes()
graph_fly.number_of_edges()

2989

4660

In [6]:
graph_fly.nodes?

Signature:   graph_fly.nodes(data=False, default=None)
Type:        NodeView
String form: ['4R79.2', 'F33A8.4', 'K08E4.1', 'Y37E3.9', 'Y59A8B.1', 'AC3.2', 'C55A6.10', 'AC3.7', 'C36B7.6',  <...> .4', 'ZK970.2', 'Y75B8A.30', 'Y77E11A.11', 'ZK892.4', 'ZK328.1', 'ZK783.4', 'ZC434.4', 'ZK721.2']
Length:      2989
File:        /opt/homebrew/anaconda3/envs/mgma/lib/python3.10/site-packages/networkx/classes/reportviews.py
Docstring:  
A NodeView class to act as G.nodes for a NetworkX Graph

Set operations act on the nodes without considering data.
Iteration is over nodes. Node data can be looked up like a dict.
Use NodeDataView to iterate over node data or to specify a data
attribute for lookup. NodeDataView is created by calling the NodeView.

Parameters
----------
graph : NetworkX graph-like class

Examples
--------
>>> G = nx.path_graph(3)
>>> NV = G.nodes()
>>> 2 in NV
True
>>> for n in NV:
...     print(n)
0
1
2
>>> assert NV & {1, 2, 3} == {1, 2}

>>> G.add_node(2, color="blue")
>>> NV[2

In [ ]:
# pos=nx.planar_layout(graph_fly)
nx.draw(graph_fly)

In [ ]:
#draw the sample subgraph in matplotlib
pos=nx.planar_layout(graph_fly_sampleDraw)
labels = nx.get_edge_attributes(graph_fly_sampleDraw,'weight')
nx.draw(graph_fly_sampleDraw,with_labels=True,pos=pos)
nx.draw_networkx_edge_labels(graph_fly_sampleDraw,pos=pos,edge_labels=labels,verticalalignment="bottom",font_size=10)
ax = plt.gca()
ax.margins(0.1)
plt.axis("on")
plt.title("Subgraph of Flies' Protein-protien Interactions")
plt.show()

————————————————————————————————————
**Graph algorithms**  
references:
https://networkx.org/documentation/networkx-1.10/reference/algorithms.centrality.html

In [ ]:
nx.degree_centrality(graph_fly_sampleDraw)

In [ ]:
nx.in_degree_centrality(graph_fly_sampleDraw)

In [ ]:
nx.betweenness_centrality(graph_fly)

In [ ]:
nx.betweenness_centrality(graph_fly_sampleDraw)

In [ ]:
#nx.current_flow_betweenness_centrality(graph_fly_sampleDraw)
#for undirected graphs only

In [ ]:
nx.closeness_centrality(graph_fly_sampleDraw)

In [ ]:
# nx.eigenvector_centrality(graph_fly_sampleDraw)
# for connected graphs only

In [ ]:
#nx.current_flow_closeness_centrality(graph_fly_sampleDraw)
#for undirected graphs only

In [ ]:
#nx.communicability(graph_fly_sampleDraw)
#for undirected graphs only

In [ ]:
nx.load_centrality(graph_fly_sampleDraw)